In [10]:
from qibo.backends import construct_backend
from qibo import hamiltonians, Circuit, gates, set_backend
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from copy import deepcopy

## Initializing circuit

In [ ]:
from qibo.hamiltonians import SymbolicHamiltonian
from qibo.symbols import X, Y, Z

class XXZ_compilation(hamiltonians.SymbolicHamiltonian):
    def __init__(self, L, delta=1.0, boundary="OBC", gateset="CNOT"):

        even_pairs = [(i, (i+1)%(L)) for i in range(0, L-1, 2)]
        odd_pairs = [(i, i+1) for i in range(1, L-1, 2)]
        if boundary == "PBC":
            odd_pairs.append((0, L-1))
        pairs = even_pairs + odd_pairs

        super().__init__(sum([X(i)*X(j) + Y(i)*Y(j) + delta*Z(i)*Z(j) for (i, j) in pairs]))

        self.H_even = SymbolicHamiltonian(sum([X(i)*X(j) + Y(i)*Y(j) + delta*Z(i)*Z(j) for (i, j) in even_pairs]))
        self.H_odd = SymbolicHamiltonian(sum([X(i)*X(j) + Y(i)*Y(j) + delta*Z(i)*Z(j) for (i, j) in odd_pairs]))
        
        self.gateset = gateset
        self.boundary = boundary
        self.even_pairs = even_pairs
        self.odd_pairs = odd_pairs
        self.pairs = pairs
        self.delta = delta
    
    def circuit(self,dt, ts_order = 1):
        c_even = Circuit(self.nqubits)
        c_odd = Circuit(self.nqubits)
        
        if ts_order == 1:
            for (i, j) in self.even_pairs:
                self.add_XXZ_term_dt(c_even, i, j, dt)            
            for (i, j) in self.odd_pairs:
                self.add_XXZ_term_dt(c_odd, i, j, dt)
            return c_even + c_odd
        elif ts_order == 2:
            for (i, j) in self.even_pairs:
                self.add_XXZ_term_dt(c_even, i, j, dt)            
            for (i, j) in self.odd_pairs:
                self.add_XXZ_term_dt(c_odd, i, j, dt/2)
            return c_odd+c_even + c_odd


    def add_XXZ_term_dt(self,qc,i,j,dt):
        if self.gateset == "CNOT":

            alpha = -dt
            beta = -dt
            gamma = -self.delta * dt

            qc.add(gates.RZ(i, -np.pi / 2))
            qc.add(gates.CNOT(i, j))
            qc.add(gates.RZ(j, -2 * gamma + np.pi / 2))
            qc.add(gates.RY(i, -np.pi / 2 + 2 * alpha))
            qc.add(gates.CNOT(j, i))
            qc.add(gates.RY(i, -2 * beta + np.pi / 2))
            qc.add(gates.CNOT(i, j))
            qc.add(gates.RZ(j, np.pi / 2))

            return qc 
        
        elif self.gateset == "RZZ":
            qc.add(gates.RZZ(i, j, dt))        

            qc.add(gates.H(i)), qc.add(gates.H(j))
            qc.add(gates.RZZ(i, j, dt))
            qc.add(gates.H(i)), qc.add(gates.H(j))        

            qc.add(gates.SDG(i)), qc.add(gates.SDG(j))
            qc.add(gates.H(i)), qc.add(gates.H(j))
            qc.add(gates.RZZ(i, j, dt*delta))
            qc.add(gates.H(i)), qc.add(gates.H(j))  
            qc.add(gates.S(i)), qc.add(gates.S(j))
        
            return qc   





[Qibo 0.2.21|INFO|2025-08-01 14:23:54]: Using qibojit (numba) backend on /CPU:0


NameError: name 'delta' is not defined

In [ ]:



def h_expectation_from_circ(ham, circ):
    # calculates the exact expectation of hamiltonian given a circuit in qibo
    return ham.expectation(
        ham.backend.execute_circuit(circuit=circ).state())
    
# XXZ model matrix
from qibo.symbols import X, Y, Z
def construct_XXZ(nqubits, delta=0.5, boundary='closed'):
    if boundary == 'periodic' or boundary == 'closed':
        return hamiltonians.XXZ(nqubits, delta)
    elif boundary == 'open':
        H_sym = sum([X(i)*X(i+1)+ Y(i)*Y(i+1) + delta* Z(i)*Z(i+1) for i in range(nqubits-1)])
        return hamiltonians.SymbolicHamiltonian(H_sym)

In [12]:
# import initialization
L = 16
delta = 1
nlayers = 1
H = construct_XXZ(L, delta, "open")
psi0 = np.zeros(2**L)
psi0[0] = 1
file_path = f"results/circuit_qasm/cobyla_{L}q_{nlayers}l_XXZ/hva_circ.qasm"
with open(file_path, 'r') as file:
            circ_str = file.read()
init_circ = Circuit.from_qasm(circ_str)
print(h_expectation_from_circ(H, init_circ))

-26.978755907262773


## Hamiltonian simulation

$$e^{-itH} \approx e^{-itH_0}e^{-itH_1}\approx e^{-it/2H_0}e^{-itH_1}e^{-it/2H_0}$$

In Qibo, `RZZ(q0, q1, theta)`$=e^{-i\frac{\theta}{2}ZZ}$

In [13]:
# check rxx formula
t = 0.5
qc = Circuit(2)
qc.add(gates.RXX(0, 1, t*2))
x = np.array([[0,1], [1,0]])
xx = np.kron(x, x)
u = sp.linalg.expm(-1j*t*xx)
print(np.linalg.norm(u-qc.unitary())<1e-3)

True


In [2]:
nqubits = 7
boundary = "closed"
even_pairs = [(i, (i+1)%(nqubits)) for i in range(0, nqubits-1, 2)]
odd_pairs = [(i, i+1) for i in range(1, nqubits-1, 2)]
if boundary == 'closed':
    odd_pairs.append((0, nqubits-1))

print(even_pairs)
print(odd_pairs)

[(0, 1), (2, 3), (4, 5)]
[(1, 2), (3, 4), (5, 6), (0, 6)]


In [ ]:
nqubits = 6
boundary = "closed"
even_pairs = [(i, (i+1)%(nqubits)) for i in range(0, nqubits-1, 2)]
odd_pairs = [(i, i+1) for i in range(1, nqubits-1, 2)]
if boundary == 'closed':
    odd_pairs.append((0, nqubits-1))

print(even_pairs)
print(odd_pairs)
qc = Circuit(G.number_of_nodes())

[(0, 1), (2, 3), (4, 5)]
[(1, 2), (3, 4), (0, 5)]


In [ ]:
def XXZ_circuit_dt(G, delta, t, trotter_steps=2, layer=3, ts_order):
    even_pairs = [e for e in G.edges() if e[0] % 2 == 0]
    odd_pairs  = [e for e in G.edges() if e[0] % 2 == 1]
    print(even_pairs)
    print(odd_pairs)
    qc = Circuit(G.number_of_nodes())
    dt_even = t/trotter_steps*2
    dt_odd = t/trotter_steps*2
    if layer == 3:
        dt_even /= 2
        
    def RZZ(i, j, t):
        qc.add(gates.RZZ(i, j, t))
        
    def RXX(i, j, t):
        qc.add(gates.H(i)), qc.add(gates.H(j))
        RZZ(i, j, t)
        qc.add(gates.H(i)), qc.add(gates.H(j))
        
    def RYY(i, j, t):
        qc.add(gates.SDG(i)), qc.add(gates.SDG(j))
        RXX(i, j, t)
        qc.add(gates.S(i)), qc.add(gates.S(j))
        
    for _ in range(trotter_steps):
        for (i,j) in even_pairs:
            RXX(i, j, dt_even)
            RYY(i, j, dt_even)
            RZZ(i, j, dt_even*delta)
        for (i, j) in odd_pairs:
            RXX(i, j, dt_odd)
            RYY(i, j, dt_odd)
            RZZ(i, j, dt_odd*delta)
        if layer == 3:
            for i,j in even_pairs:
                RXX(i, j, dt_even)
                RYY(i, j, dt_even)
                RZZ(i, j, dt_even*delta)
                
    return qc

In [55]:
# check hamiltonian simulation
import networkx as nx
L_test = 6
delta_test = 0.5
G = nx.Graph()
# G.add_edges_from([(k, (k+1)%L) for k in range(L)]) # periodic boundary condition
G.add_edges_from([(k, (k+1)%L_test) for k in range(L_test-1)]) # open boundary
boundary_test = 'open'
H_test = construct_XXZ(L_test, delta_test, boundary_test)
t = 0.2
print(np.linalg.norm(sp.linalg.expm(-1j*t*H_test.matrix) - XXZ_simulation(G, delta_test, t, layer=3).unitary()))

[Qibo 0.2.21|WARNING|2025-08-01 14:09:07]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.21|ERROR|2025-08-01 14:09:07]: Attempting to add gate with target qubits (5,) on a circuit of 5 qubits.


[(0, 1), (2, 3), (4, 5)]
[(1, 2), (3, 4)]


ValueError: Attempting to add gate with target qubits (5,) on a circuit of 5 qubits.

## Magnetic field

RZ in qibo: $e^{-i\frac{\theta}{2}D}$

In [17]:
# magnetic field
def magnetic_field_circ(coefs, t):
    # implments e^{-itD}
    nqubits = len(coefs)
    qc = Circuit(nqubits)
    
    for idx, coef in enumerate(coefs):
        qc.add(gates.RZ(idx, coef*t*2))
    return qc 

In [18]:
# test d formula
theta = 0.5
d = magnetic_field_circ([1], theta)
d_u = sp.linalg.expm(-1j*theta*np.array([[1,0],[0,-1]]))
print(np.linalg.norm(d.unitary()-d_u)<1e-3)

True


## Construct GCI circuit
In the initialization step, we have
$$
\bra{\psi_0(\theta)} H\ket{\psi_0(\theta)}=\bra{0}U_\theta^\dagger H U_\theta\ket{0}=\bra{0}A_0\ket{0}
$$
For one step of DBQA, 
$$
V_1 = e^{itD}U^\dagger e^{-itH_0}Ue^{-itD}
$$
and that
$$
A_1 = V_1^\dagger U^\dagger H_0 U V_1.
$$

With this we find
$$
\langle 0|A_1|0\rangle =\langle \psi_1|H|\psi_1\rangle = \bra 0 U_1^\dagger H U_1\ket 0
$$
where 
$$
|\psi_1\rangle = U* V_1|0\rangle.
$$

Therefore, our composed DBQA circuit should follow

$$
U_1 = U*V_1 = U_\theta e^{itD}U_\theta ^\dagger e^{-itH}U_\theta e^{-itD}
$$

$$
U_1^{(RHOPF)} = U \cdot V_1^{(RHOPF)}=Ue^{-i\phi \sqrt t D}U^\dagger e^{-i\sqrt tH} U
            e^{i(\phi+1)\sqrt t D}U^\dagger e^{i(1-\phi)\sqrt tH} U e^{-i\phi \sqrt t D}.
$$

In [19]:
def GCI(H, H_sim_fun, D_circ_fun, init_circ, t):
    qc = Circuit(H.nqubits)
    H_sim = H_sim_fun(t)
    D_circ = D_circ_fun(t)
    qc.wire_names = init_circ.wire_names
    D_circ.wire_names = qc.wire_names
    H_sim.wire_names = qc.wire_names
    qc = qc + D_circ + init_circ + H_sim + init_circ.invert() + D_circ.invert() + init_circ
    return qc, h_expectation_from_circ(H, qc)

In [20]:
# test t=0 same value as E_init
t = 0
coefs = [1] * L
H_sim_fun = lambda t: XXZ_simulation(L, delta, t)
D_circ_fun = lambda t: magnetic_field_circ(coefs, t)
gci_circ, gci_val = GCI(H, H_sim_fun, D_circ_fun, init_circ, t)
print(np.abs(gci_val-h_expectation_from_circ(H, init_circ))<1e-3)

True


In [21]:
run_grid_search = False
if run_grid_search:
    t_space = np.linspace(0,0.3,20)
    E_ls = []
    for t in t_space:
        gci_circ, gci_val = GCI(H, H_sim_fun, D_circ_fun, init_circ, t)
        E_ls.append(gci_val)

In [22]:
if run_grid_search:
    E_min = min(E_ls)
    min_idx = E_ls.index(E_min)
    t_min = t_space[min_idx]
    plt.plot(t_space, E_ls)
    plt.scatter(t_min, E_min, color='red', label=f"({t_min:.2f}, {E_min:.2f})")
    plt.title(f'DBQA gain with time - XXZ (L={L})')
    plt.xlabel('time')
    plt.ylabel(r'$\langle H\rangle$')
    plt.legend()

In [23]:
gci_circ.gate_names

Counter({'h': 752,
         'rzz': 273,
         'sdg': 182,
         's': 182,
         'x': 48,
         'rz': 32,
         'cx': 24})

In [24]:
print("Circuit depth:", gci_circ.depth)
print("Circuit total gate count:", gci_circ.ngates)
print("Circuit CX count:", gci_circ.gate_names['cx'])
print("Estimated equivalent CZ count:", gci_circ.gate_names['cx']+
      gci_circ.gate_names['rxx']+gci_circ.gate_names['ryy']+gci_circ.gate_names['rzz']
      )

Circuit depth: 119
Circuit total gate count: 1493
Circuit CX count: 24
Estimated equivalent CZ count: 297


## Optimize parameters

### Fixed D optimize t

Here we expect the same value as the grid search but faster run time.

In [25]:
from scipy.optimize import minimize_scalar
coefs = [1] * L
def objective(t):
    H_sim_fun = lambda t: XXZ_simulation(L, delta, t)
    D_circ_fun = lambda t: magnetic_field_circ(coefs, t)
    gci_circ, gci_val = GCI(H, H_sim_fun, D_circ_fun, init_circ, t)
    return gci_val

In [26]:
res = minimize_scalar(
    objective,                  
    method='brent',    
    options={'maxiter': 15,
             'xtol': 1e-3} 
)
print(res.fun)
print(res.x)

-27.327291516176324
0.14351650971650587


### Optimize both D and t

In [27]:
from scipy.optimize import minimize
coefs = [1] * L
D_s = coefs + [0.1]
def objective(D_s):
    coefs = D_s[:-1]
    t = D_s[-1]
    H_sim_fun = lambda t: XXZ_simulation(L, delta, t)
    D_circ_fun = lambda t: magnetic_field_circ(coefs, t)
    gci_circ, gci_val = GCI(H, H_sim_fun, D_circ_fun, init_circ, t)
    return gci_val

In [28]:
max_iter = 1000
result = minimize(
    objective,
    D_s,
    method="COBYLA",
    options={"disp": True, "maxiter": max_iter},
    tol=1e-2,
)

print(result.fun)
print(result.x)

Return from COBYLA because the trust region radius reaches its lower bound.
Number of function values = 111   Least value of F = -27.30661157305762
The corresponding X is:
[1.94500009 1.94499969 1.94500089 1.94500025 1.94500069 1.9450007
 1.94500039 1.94499948 1.94500012 1.94499972 1.9449998  1.94500016
 1.99499992 1.01749995 1.99499976 1.99499997 0.09955262]

-27.30661157305762
[1.94500009 1.94499969 1.94500089 1.94500025 1.94500069 1.9450007
 1.94500039 1.94499948 1.94500012 1.94499972 1.9449998  1.94500016
 1.99499992 1.01749995 1.99499976 1.99499997 0.09955262]
